In [1]:
from utils import create_facts_and_examples
import numpy as np
import pandas as pd
import random, os
np.random.seed(0) #Code B

### create G.19


In [2]:
a=1
b=2
c=3
d=4
df = pd.DataFrame({'X':[a,b,c,b,a,a,a,b,b,b], 
                   'Y':[b,c,d,a,c,d,a,d,a,b]})
df['Fraud--X--Y'] = True
df['Transaction--X--Y'] = df['X']!=df['Y']

In [3]:
create_facts_and_examples(df_= df, target='Fraud--X--Y', 
                          predicates=['Transaction--X--Y'], 
                          output_dir="examples/G.19")

Fraud--X--Y 1 2 flag= True
Fraud--X--Y 2 3 flag= True
Fraud--X--Y 3 4 flag= True
Fraud--X--Y 2 1 flag= True
Fraud--X--Y 1 3 flag= True
Fraud--X--Y 1 4 flag= True
Fraud--X--Y 1 1 flag= True
Fraud--X--Y 2 4 flag= True
Fraud--X--Y 2 1 flag= True
Fraud--X--Y 2 2 flag= True
Transaction--X--Y
Transaction--X--Y 1 2 flag= True
Transaction--X--Y 2 3 flag= True
Transaction--X--Y 3 4 flag= True
Transaction--X--Y 2 1 flag= True
Transaction--X--Y 1 3 flag= True
Transaction--X--Y 1 4 flag= True
Transaction--X--Y 2 4 flag= True
Transaction--X--Y 2 1 flag= True


In [4]:
N = 10

customer = ['c'+str(i) for i in range(5)]
Fraud = np.random.randint(2, size=N)
df = pd.DataFrame({'orig':np.random.choice(customer, size=N), 
                   'destination':np.random.choice(customer, size=N),\
                   "Fraud":Fraud})


In [5]:
df.drop(df[df['orig']==df['destination']].index, inplace=True)

In [6]:
fraudsters = df[df['Fraud']==True]

In [7]:
fraudsters[['orig','destination']]

,orig,destination
1,c4,c1
2,c0,c4
4,c4,c0
5,c2,c3
6,c1,c0
7,c0,c2
8,c1,c3
9,c1,c0


In [8]:
index_from_merge = pd.merge(df.reset_index(), fraudsters[['destination']].drop_duplicates(), how='inner', left_on='orig', right_on='destination')

In [9]:
index_from_merge

,index,orig,destination_x,Fraud,destination_y
0,0,c2,c0,0,c2
1,5,c2,c3,1,c2
2,1,c4,c1,1,c4
3,4,c4,c0,1,c4
4,2,c0,c4,1,c0
5,3,c0,c3,0,c0
6,7,c0,c2,1,c0
7,6,c1,c0,1,c1
8,8,c1,c3,1,c1
9,9,c1,c0,1,c1


In [10]:
df['Fraud_chain--orig--destination']=0
df.loc[index_from_merge['index'],'Fraud']=1
df.loc[index_from_merge['index'],'Fraud_chain--orig--destination']=1
df['From--orig']=1
df['To--destination']=1

In [11]:
df.loc[index_from_merge['index'],:]

,orig,destination,Fraud,Fraud_chain--orig--destination,From--orig,To--destination
0,c2,c0,1,1,1,1
5,c2,c3,1,1,1,1
1,c4,c1,1,1,1,1
4,c4,c0,1,1,1,1
2,c0,c4,1,1,1,1
3,c0,c3,1,1,1,1
7,c0,c2,1,1,1,1
6,c1,c0,1,1,1,1
8,c1,c3,1,1,1,1
9,c1,c0,1,1,1,1


### creating fact, positive and negative examples

In [12]:
#drop first letter and convert to int
df['orig'] = df['orig'].str[1:].astype('int')
df['destination'] = df['destination'].str[1:].astype('int')

In [13]:
df['Fraud--orig--destination']=df['Fraud']
df['Transfer--orig--destination']=1
create_facts_and_examples(df_= df, target='Fraud_chain--orig--destination', 
                          predicates=['Fraud--orig--destination', 'Transfer--orig--destination'], 
                          output_dir="examples/debug-simple-transitivity")

Fraud_chain--orig--destination 2 0 flag= True
Fraud_chain--orig--destination 4 1 flag= True
Fraud_chain--orig--destination 0 4 flag= True
Fraud_chain--orig--destination 0 3 flag= True
Fraud_chain--orig--destination 4 0 flag= True
Fraud_chain--orig--destination 2 3 flag= True
Fraud_chain--orig--destination 1 0 flag= True
Fraud_chain--orig--destination 0 2 flag= True
Fraud_chain--orig--destination 1 3 flag= True
Fraud_chain--orig--destination 1 0 flag= True
Fraud--orig--destination
Fraud--orig--destination 2 0 flag= True
Fraud--orig--destination 4 1 flag= True
Fraud--orig--destination 0 4 flag= True
Fraud--orig--destination 0 3 flag= True
Fraud--orig--destination 4 0 flag= True
Fraud--orig--destination 2 3 flag= True
Fraud--orig--destination 1 0 flag= True
Fraud--orig--destination 0 2 flag= True
Fraud--orig--destination 1 3 flag= True
Fraud--orig--destination 1 0 flag= True
Transfer--orig--destination
Transfer--orig--destination 2 0 flag= True
Transfer--orig--destination 4 1 flag= True
T